<a href="https://colab.research.google.com/github/archanaanbalagan11/Codsoft/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving movie_recommendation_dataset.zip to movie_recommendation_dataset.zip
User uploaded file "movie_recommendation_dataset.zip" with length 579 bytes


In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
with zipfile.ZipFile("movie_recommendation_dataset.zip", "r") as zip_ref:
    zip_ref.extractall()
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')
merged_df = pd.merge(ratings_df, movies_df, on='movieId')
user_item_matrix = merged_df.pivot_table(index='userId', columns='title', values='rating').fillna(0)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_recommendations(user_id, user_item_matrix, user_similarity_df, num_recommendations=5):
    """
    Generates movie recommendations for a user based on collaborative filtering.

    Args:
        user_id: The ID of the user for whom to generate recommendations.
        user_item_matrix: The user-item matrix.
        user_similarity_df: The user-user similarity DataFrame.
        num_recommendations: The number of recommendations to generate.

    Returns:
        A list of recommended movie titles.
    """
    user_similarity_scores = user_similarity_df.loc[user_id]
    sorted_similar_users = user_similarity_scores.sort_values(ascending=False)
    user_rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    predicted_ratings = {}
    for similar_user_id in sorted_similar_users.index:
        if similar_user_id != user_id:
            similar_user_rated_movies = user_item_matrix.loc[similar_user_id][user_item_matrix.loc[similar_user_id] > 0].index
            unrated_movies = similar_user_rated_movies.difference(user_rated_movies)

            for movie in unrated_movies:
                similar_user_rating = user_item_matrix.loc[similar_user_id, movie]
                similarity_score = sorted_similar_users.loc[similar_user_id]
                if movie not in predicted_ratings:
                    predicted_ratings[movie] = 0
                predicted_ratings[movie] += similar_user_rating * similarity_score
    sorted_recommendations = sorted(predicted_ratings.items(), key=lambda item: item[1], reverse=True)
    return [movie for movie, rating in sorted_recommendations[:num_recommendations]]
user_id_to_recommend = 1
recommendations = get_recommendations(user_id_to_recommend, user_item_matrix, user_similarity_df)
print(f"Recommendations for user {user_id_to_recommend}: {recommendations}")

Recommendations for user 1: ['Father of the Bride Part II (1995)', 'Waiting to Exhale (1995)']
